## For loading super symmetry data
Written originally by Kabir S., adapted by John C.

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score

In [57]:

data = pd.read_csv("supersymmetry_dataset.csv")
display(data)

,SUSY,lepton 1 pT,lepton 1 eta,lepton 1 phi,lepton 2 pT,lepton 2 eta,lepton 2 phi,missing energy magnitude,missing energy phi,MET_rel,axial MET,M_R,M_TR_2,R,MT2,S_R,M_Delta_R,dPhi_r_b,cos(theta_r1)
0,0.0,0.707258,0.613159,-0.763354,1.010814,1.672101,1.544314,0.342641,0.286944,0.514347,-0.008000,0.832800,0.646217,0.688581,1.159059,0.807485,0.890080,0.272718,0.014631
1,0.0,0.493745,-1.745581,-0.640968,0.576161,-0.514477,-0.629788,0.585841,0.874038,0.879419,-0.221701,0.553944,0.770422,1.234180,1.671171,0.511589,1.255531,0.521592,0.158222
2,1.0,1.305413,0.283820,-1.149154,0.507217,-0.194161,1.313993,0.659832,0.619593,0.534922,-0.320298,0.861971,1.085602,1.117620,1.199941,0.754778,0.984459,0.235892,0.601631
3,1.0,0.809715,-0.358303,-0.497172,0.787675,0.027419,1.201305,0.790556,-0.861923,0.740287,0.890525,0.684771,0.717640,0.929989,0.341225,0.683829,0.492741,1.283829,0.115851
4,0.0,0.912852,1.797455,-1.360165,0.496723,1.199644,0.627462,1.206452,-1.362983,0.033791,1.509121,0.659545,0.652570,0.878010,0.000000,0.580894,0.143379,1.438492,0.230821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,0.0,0.442663,-1.081974,1.586361,0.743882,-1.523730,0.911325,1.163068,-0.281503,1.745908,-0.778513,0.481283,1.072933,1.978277,2.230672,0.587822,1.677162,1.321179,0.180186
4999996,1.0,1.703428,-0.642585,1.197606,0.656566,1.518905,-0.168589,1.111145,0.496978,1.550931,0.813373,1.820490,1.014322,0.494429,0.000000,1.709095,1.067639,0.724796,0.109009
4999997,1.0,3.787456,-0.456186,-0.536557,1.556609,1.263663,1.132305,3.464473,1.236886,0.922248,-3.164055,3.452112,4.078378,1.048381,0.670448,3.584134,4.127820,1.557259,0.373298
4999998,1.0,1.584892,-0.644660,-0.287854,2.301142,-0.440804,1.359573,0.240126,-1.668636,0.254883,0.076701,1.554480,0.704920,0.402412,0.472014,1.558838,0.357319,0.295405,0.070806


In [38]:
# In[3]:


n = 800# Data_Tested for first 800 points
data_trimmed = data[:n]
#display(data_trimmed)

In [39]:


import itertools
import pandas as pd

# Given list of 19 feature names
features = ['lepton 1 pT', 'lepton 1 eta', 'lepton 1 phi', 'lepton 2 pT',
            'lepton 2 eta', 'lepton 2 phi', 'missing energy magnitude',
            'missing energy phi', 'MET_rel', 'axial MET', 'M_R', 'M_TR_2', 'R',
            'MT2', 'S_R', 'M_Delta_R', 'dPhi_r_b', 'cos(theta_r1)']

# Generate all unique 3-feature combinations (19C3)
combinations_list = list(itertools.combinations(features, 3))

In [41]:
columns = data_trimmed.columns
a_max = []
for p , q , r in combinations_list:
    X = data_trimmed.iloc[:n][[p, q,r]].values 
    y = data_trimmed["SUSY"]
    X_train , X_test , y_train, y_test = train_test_split(X , y ,test_size = 0.2, random_state = 3)
    weights = ["uniform"]# I am gonna test for these weights
    neighbors = [1,2,3,4,5,6,7,8,9,10,12,12,13,14,15,20,25,30,35,40,45,50,100]# No of K Neighbors , I couldnt vary more as it was taking a lot of time
    l = []
    for i in weights:
        for j in neighbors:
            model = KNN(j , weights = i)
            model.fit(X_train , y_train)
            y_pred = model.predict(X_test)
            y_true = y_test
            a = accuracy_score(y_true ,y_pred)
            cvs = cross_val_score(model , X_train ,y_train)
            cvsm = np.mean(cvs)
            l.append([j , i ,a , cvsm])

    df_final = pd.DataFrame(l , columns = ["K" , "Weights" , 'Accuracy' , "CVS Mean"])
    a_max.append([p,q,r,np.max(df_final.Accuracy),df_final.loc[df_final['Accuracy'].idxmax(), 'K']])


In [42]:
df_accuracy = pd.DataFrame(a_max , columns = ["Var1" , "Var2" , "Var3" , "Max_Accuracy", "K"])
display(df_accuracy.describe())
print(df_accuracy)

,Max_Accuracy,K
count,816.000000,816.000000
mean,0.695190,27.159314
std,0.047471,27.787522
min,0.525000,1.000000
25%,0.662500,8.000000
50%,0.706250,15.000000
75%,0.731250,40.000000
max,0.787500,100.000000


            Var1          Var2                      Var3  Max_Accuracy   K
0    lepton 1 pT  lepton 1 eta              lepton 1 phi       0.73750   6
1    lepton 1 pT  lepton 1 eta               lepton 2 pT       0.69375  12
2    lepton 1 pT  lepton 1 eta              lepton 2 eta       0.69375  30
3    lepton 1 pT  lepton 1 eta              lepton 2 phi       0.70000  12
4    lepton 1 pT  lepton 1 eta  missing energy magnitude       0.76875  25
..           ...           ...                       ...           ...  ..
811          MT2      dPhi_r_b             cos(theta_r1)       0.60000   2
812          S_R     M_Delta_R                  dPhi_r_b       0.67500  25
813          S_R     M_Delta_R             cos(theta_r1)       0.68750   3
814          S_R      dPhi_r_b             cos(theta_r1)       0.70000  15
815    M_Delta_R      dPhi_r_b             cos(theta_r1)       0.67500  12

[816 rows x 5 columns]


In [44]:
display(df_accuracy[df_accuracy.Max_Accuracy==max(df_accuracy["Max_Accuracy"])])# Gives the list of all potential optimal features

,Var1,Var2,Var3,Max_Accuracy,K
19,lepton 1 pT,lepton 1 phi,missing energy magnitude,0.7875,4
46,lepton 1 pT,lepton 2 eta,missing energy magnitude,0.7875,30
79,lepton 1 pT,missing energy magnitude,dPhi_r_b,0.7875,100


In [60]:
import pickle

data = data[:5000]

c1 = data["lepton 1 pT"].to_numpy()
c2 = data["lepton 1 phi"].to_numpy()
c3 = data["missing energy magnitude"].to_numpy()

values = np.stack([c1,c2,c3], axis=1)
print(values)
labels = data["SUSY"]

data_to_output = {"values":values, "labels":labels}

with open('data.pickle', 'wb') as handle:
    pickle.dump(data_to_output, handle, protocol=pickle.HIGHEST_PROTOCOL)



[[ 0.70725811 -0.76335365  0.34264132]
 [ 0.49374521 -0.64096761  0.58584052]
 [ 1.30541348 -1.14915431  0.65983236]
 ...
 [ 1.88443446  1.26519644  1.55331123]
 [ 0.67102247 -1.19830644  1.47911882]
 [ 0.62559527 -0.9227156   0.52267367]]
